In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [4]:
from io import StringIO

In [5]:
def secondsConverter(time: str):
  minutes, seconds_ms = time.split(":")
  minutes_to_seconds = int(minutes) * 60
  seconds, miliseconds = seconds_ms.split(".")
  total_seconds = (int(miliseconds) / 1000) + int(seconds) + minutes_to_seconds 
  return total_seconds

In [6]:
all_dataframes = None # variable that will hold the merged dataframes
list_dataframes = []
  

In [7]:
# iterate through 5 years (2020-2025) and merge the corresponding data frames into one
for i in range(6):
  # get the data frame from rankings 
  year = str(2025-i)
  url = "https://worldathletics.org/records/toplists/middlelong/1500-metres/all/men/senior/" + year + "?regionType=world&page=1&bestResultsOnly=true&maxResultsByCountry=all&eventId=10229502&ageCategory=senior"
  response = requests.get(url, verify=False)
  all_players = pd.read_html(StringIO(response.text))

  # cleaning data with pandas
  curr_table = all_players[0]
  curr_table = curr_table.drop(columns=["WIND","Unnamed: 7", "Pos"])
  curr_table["Mark"] = curr_table["Mark"].apply(secondsConverter) # convert 1500 time to total seconds

  try : # not all players have a recorded DOB
    curr_table["DOB"] = pd.to_datetime(curr_table["DOB"], format="%d %b %Y", errors="coerce").dt.date
  except ValueError:
    pass
  # rename misleading columns
  curr_table = curr_table.rename(columns= { 
    "DOB": "Date_of_Birth",
    "Nat": "Nationality",
    "Results Score": "Results_Score"
  })

  # add a year column
  curr_table["Year"] = year

  # add dataframe to a list of dataframes
  list_dataframes.append(curr_table)
  
# finally merge all dataframes
all_dataframes = pd.concat(list_dataframes)
  

c:\Users\gabin\VSCode_Projects\Python_Projects\.conda\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'worldathletics.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\gabin\VSCode_Projects\Python_Projects\.conda\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'worldathletics.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\gabin\VSCode_Projects\Python_Projects\.conda\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'worldathletics.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usa

In [8]:
all_dataframes.to_csv("Athletics_rankings.csv", index=False)